In [ ]:
import numpy as np
from findpeaks import findpeaks

In [ ]:
import numpy as np
from scipy.io import readsav
import matplotlib.pyplot as plt
import cv2
from pathlib import Path
from tempfile import TemporaryFile

from matplotlib.animation import FuncAnimation
from matplotlib.widgets import Slider
import imageio

In [ ]:
def _load_data(filename):
    dat = readsav(filename)
    emission = dat['emission_structure']
    return emission[0]

def _find_index(arr,val):
    return np.argmin(abs(arr-val))

In [ ]:
def norm(data):
    mn = data.mean()
    std = data.std()
    return((data-mn)/std)

def rescale(data):
    return (data-data.min())/(data.max()-data.min())

def quantfilt(src,thr=0.9):
    filt = np.quantile(src,thr,axis=0)
    out = np.where(src<filt,0,src)
    return out

# gaussian filtering
def gaussblr(src,filt=(31, 3)):
    src = (rescale(src)*255).astype('uint8')
    out = cv2.GaussianBlur(src,filt,0)
    return rescale(out)

# mean filtering
def meansub(src):
    mn = np.mean(src,axis=1)[:,np.newaxis]
    out = np.absolute(src - mn)
    return rescale(out)

# morphological filtering
def morph(src):
    src = (rescale(src)*255).astype('uint8')
    se1 = cv2.getStructuringElement(cv2.MORPH_RECT, (4,4))
    se2 = cv2.getStructuringElement(cv2.MORPH_RECT, (3,1))
    mask = cv2.morphologyEx(src, cv2.MORPH_CLOSE, se1)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, se2)
    return rescale(mask)

In [ ]:
[inverted,radii,elevation,frames,times,vid_frames,vid_times,vid] = _load_data('emission_structure_pu_cam240perp_185821.sav')

200 - points above
210 - 2 points

In [ ]:
fid = 131
tid = _find_index(vid_times,times[fid]) #find frame id for camera image with t=times[fid]
find_x = inverted[fid]

In [ ]:
img = inverted[fid].copy()
gray=(255-255*(img-np.min(img))/(np.max(img)-np.min(img))).astype('uint8')
useHarrisDetector = False # False uses Shi-Tomasi Corner Detector
corners = np.intp(cv2.goodFeaturesToTrack(gray,3,.5,10, useHarrisDetector=useHarrisDetector))
x = radii[fid][corners[:,0,0]]
y = elevation[fid][corners[:,0,1]]
print(np.column_stack((x,y)))
# plt.imshow(img)
plt.pcolormesh(radii[fid],elevation[fid],img,shading='auto', cmap='gray')
plt.scatter(x,y,color='red')
plt.title(f"Inverted with Radiation Points, Frame {fid}")
plt.show()

In [ ]:
def get_point(img, radii, elevation):
    gray=(255-255*(img-np.min(img))/(np.max(img)-np.min(img))).astype('uint8')
    corners = cv2.goodFeaturesToTrack(gray,3,0.5,10, useHarrisDetector=False)
    corners = np.intp(corners)
    x = radii[corners[:,0,0]]
    y = elevation[corners[:,0,1]]
    return np.column_stack((x, y))

def check_points(points):
    return_points = []
    
    if (points.shape[0] == 1) or (points.shape[0] == 2):
        return points
        
    if points.shape[0] == 3:
        x_sort = sorted(points, key=lambda point: point[0])
        if x_sort[0][1] < x_sort[1][1]:
            return_points.append(x_sort[0])
        if x_sort[2][1] < x_sort[1][1]:
            return_points.append(x_sort[2])
        return_points.append(x_sort[1])
    
    return return_points

def process_points(inverted, radii, elevation):
    points = []
    for fid in range(inverted.shape[0]):
        temp_points = get_point(inverted[fid], radii[fid], elevation[fid])
        temp_points = check_points(temp_points)
        # temp_points = sorted(temp_points, key=lambda point: point[0])
        points.append(temp_points)
        
    return points
        

In [ ]:
potential_points = process_points(inverted, radii, elevation)

In [ ]:
color_dict = {0:'red', 1:'blue', 2:'brown', 3:'yellow', 4:'orange', 5:'purple', 6:'pink'}

def update_image(img, idx):
    plt.clf()
    img = inverted[idx].copy()
    x_point,y_point = zip(*potential_points[idx])
    point_color = [color_dict[i] for i in range(len(potential_points[idx]))]
    point_label = [i for i in range(len(potential_points[idx]))]
    plt.pcolormesh(radii[idx],elevation[idx],img,shading='auto')
    plt.scatter(x_point,y_point,color=point_color,label=point_label)
    
    markers = [plt.Line2D([0,0],[0,0],color=color, marker='o', linestyle='') for color in point_color]
    plt.legend(markers, point_label, numpoints=1)
    plt.title(f'Frame {idx}')
    plt.show()

In [ ]:
update_image(inverted, 201)

In [ ]:
path = Path('output_images')
for fid in range(times.size):
    new_path = path / f'{fid}.png'
    plt.pcolormesh(radii[fid],elevation[fid],inverted[fid],shading='auto')
    # plt.scatter(x_point[fid,0],x_point[fid,1],color='red')
    plt.savefig(new_path,bbox_inches='tight')